In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier,plot_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix

import re
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from wordcloud import WordCloud

import PyPDF2


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\munge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\munge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
loaded_model = pickle.load(open('xgb_pro_job.pkl', 'rb')) 
loaded_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [9]:
resumeDataSet = pd.read_csv('resumeDataSet.csv')
# display(resumeDataSet.head())

requiredText = resumeDataSet['cleaned_resume'].values
requiredTarget = resumeDataSet['Category'].values
# print('requiredText : ',requiredText,'\n')
# print('requiredTarget : ',requiredTarget,'\n')

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_features=1500)
word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)
# print('WordFeatures : ',WordFeatures,'\n')

print ("Feature completed .....")

# test_text = [t1] # Test_CVs['cleaned_resume'].values

# # word_vectorizer = TfidfVectorizer(
# #     sublinear_tf=True,
# #     stop_words='english',
# #     max_features=1500)
# # word_vectorizer.fit(test_text)

# WordFeatures = word_vectorizer.transform(test_text)
# print('WordFeatures : \n',WordFeatures)

Feature completed .....


In [10]:

pdfFileObj = open('Resume/1639482865701.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

text_data = '' 
for ind in range(pdfReader.numPages):
#     print(ind)
    
    pageObj = pdfReader.getPage(ind)
    data1=pageObj.extractText()
#     print(data1)
    text_data = text_data + data1
    
pdfFileObj.close()
    
text_data

"Adams \nAllen\n \n+91 \n0000000000\n \n| \nabc\n@gmail.com | Mumbai | /in/\nxxx\n-\nxxx\n/\n \nBusiness Correspondent | Political Correspondent | Digital Journalist\n \nProfessional Summary\n \n \nCreative and driven journalist with 5+ years of \nprofessional \nexperience. Great storytelling skills and \na nose for the truth as \nwell \nan excellent track record as a staff reporter in print and digital media. Associated with various government sector \ncompanie\ns, ministries, state run organizations and government bodies across \nxxx\n. Seeking to leverage skills and \nexperience to contribute to high quality journalism within India\n \nKey Areas\n \nBusiness Development, Political Relations, Reporting from Mantralaya Mumbai, Government relations, Guest coordination, \nContent \nwriting, \nwriting\n \nfor social media\n \nWork History\n \n \nBusiness Correspondent\n \nxxx\n \n| \nFeb' 2021 \n\n \nPresent\n \n\n \nReporting business stories from\n \nMantralaya in order to develop busi

In [11]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

t1 = cleanResume(text_data)
t1

'Adams Allen 91 0000000000 abc Mumbai in xxx xxx Business Correspondent Political Correspondent Digital Journalist Professional Summary Creative and driven journalist with 5 years of professional experience Great storytelling skills and a nose for the truth as well an excellent track record as a staff reporter in print and digital media Associated with various government sector companie s ministries state run organizations and government bodies across xxx Seeking to leverage skills and experience to contribute to high quality journalism within India Key Areas Business Development Political Relations Reporting from Mantralaya Mumbai Government relations Guest coordination Content writing writing for social media Work History Business Correspondent xxx Feb 2021 Present Reporting business stories from Mantralaya in order to develop business communication with various ministries Developing contents related to Agriculture and rural sector Meeting and communicating with Agriculture business 

In [12]:

test_text = [t1] # Test_CVs['cleaned_resume'].values

# word_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     stop_words='english',
#     max_features=1500)
# word_vectorizer.fit(test_text)

WordFeatures = word_vectorizer.transform(test_text)
print('WordFeatures : \n',WordFeatures)

WordFeatures : 
   (0, 1498)	0.05462668287406391
  (0, 1493)	0.12369758142340848
  (0, 1492)	0.06821691349225023
  (0, 1491)	0.0856979983542639
  (0, 1484)	0.07966205382675613
  (0, 1483)	0.03319629845208795
  (0, 1473)	0.042145430093373466
  (0, 1451)	0.06199507258612235
  (0, 1446)	0.076873091112219
  (0, 1396)	0.046764350096491235
  (0, 1390)	0.050734597207508086
  (0, 1358)	0.05891498320939219
  (0, 1338)	0.06821691349225023
  (0, 1328)	0.06854546201861722
  (0, 1323)	0.07534370654874947
  (0, 1301)	0.09486352629971143
  (0, 1292)	0.09868996900605054
  (0, 1284)	0.1244324377123846
  (0, 1274)	0.07103522969841719
  (0, 1271)	0.056197086906304095
  (0, 1265)	0.12369758142340848
  (0, 1260)	0.05884545074648726
  (0, 1244)	0.05636212881740661
  (0, 1237)	0.06956964496684999
  (0, 1232)	0.05555199305925502
  :	:
  (0, 289)	0.07439337595450653
  (0, 271)	0.0726342255781835
  (0, 269)	0.10425301858346789
  (0, 268)	0.06789439372066397
  (0, 258)	0.0335550218088545
  (0, 245)	0.11550127474

In [13]:
WordFeatures.shape

(1, 1500)

In [15]:
X_test2=WordFeatures
y_pred2 = loaded_model.predict(X_test2)
y_pred2

array([4])

In [19]:
resumeDataSet = pd.read_csv('dataset/UpdatedResumeDataSet.csv' ,encoding='utf-8')
var_mod = ['Category']
le = LabelEncoder()
for i in var_mod:
    resumeDataSet[i] = le.fit_transform(resumeDataSet[i])
# print ("CONVERTED THE CATEGORICAL VARIABLES INTO NUMERICALS")
# print('resumeDataSet : ')
# resumeDataSet

In [20]:
prediction= le.inverse_transform(y_pred2)
prediction

array(['Business Analyst'], dtype=object)

In [31]:
y_pred_1 = loaded_model.predict_proba(X_test2)
li = list((y_pred_1*100)[0])
print(list((y_pred_1*100)[0]), (y_pred_1*100)[0][y_pred2[0]])

[1.9691771, 3.9415262, 4.999068, 3.6352103, 11.063011, 3.3484497, 2.9400988, 3.4185786, 3.7576382, 3.7448828, 3.854957, 3.3922248, 4.0143256, 3.9664354, 3.2252734, 2.6033823, 3.8630333, 3.6618125, 6.1640034, 3.581186, 3.1516314, 2.9457014, 3.2234604, 2.006224, 7.528712] 11.063011


In [34]:
print(list(le.classes_), list(le.classes_)[li.index(max(li))])

['Advocate', 'Arts', 'Automation Testing', 'Blockchain', 'Business Analyst', 'Civil Engineer', 'Data Science', 'Database', 'DevOps Engineer', 'DotNet Developer', 'ETL Developer', 'Electrical Engineering', 'HR', 'Hadoop', 'Health and fitness', 'Java Developer', 'Mechanical Engineer', 'Network Security Engineer', 'Operations Manager', 'PMO', 'Python Developer', 'SAP Developer', 'Sales', 'Testing', 'Web Designing'] Business Analyst


In [35]:
df = pd.read_csv('dataset/UpdatedResumeDataSet.csv' ,encoding='utf-8')
df1 = pd.read_csv('resumeDataSet.csv')

In [36]:
df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [37]:
df1

,Category,Resume,cleaned_resume
0,6,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas num...
1,6,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details May 2013 to May 2017 B E UIT...
2,6,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...
3,6,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,6,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details MCA YMCAUST Faridabad Haryan...
...,...,...,...
957,23,Computer Skills: â¢ Proficient in MS office (...,Computer Skills Proficient in MS office Word B...
958,23,â Willingness to accept the challenges. â ...,Willingness to a ept the challenges Positive ...
959,23,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",PERSONAL SKILLS Quick learner Eagerness to lea...
960,23,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...


In [38]:
df1['Category_name'] = df['Category']
df1

,Category,Resume,cleaned_resume,Category_name
0,6,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas num...,Data Science
1,6,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details May 2013 to May 2017 B E UIT...,Data Science
2,6,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...,Data Science
3,6,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills R Python SAP HANA Tableau SAP HANA SQL ...,Data Science
4,6,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details MCA YMCAUST Faridabad Haryan...,Data Science
...,...,...,...,...
957,23,Computer Skills: â¢ Proficient in MS office (...,Computer Skills Proficient in MS office Word B...,Testing
958,23,â Willingness to accept the challenges. â ...,Willingness to a ept the challenges Positive ...,Testing
959,23,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",PERSONAL SKILLS Quick learner Eagerness to lea...,Testing
960,23,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...,Testing


In [39]:
df1.to_csv('resume.csv', index=False)